In [48]:
import os
# os.chdir('..')
import supervised 
from supervised import *

In [49]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

import pickle

In [50]:
# Read from extracted data
data = pd.read_csv('/Users/ava_mac/Documents/Coding/LHL/W6/Predicting_Flight_Delays_Midterm/data/flights_modeling_2022-10-26_2322.csv')

In [51]:
# Take copy to measure data loss after clean-up
df = data.copy()

In [52]:
# df = df.drop(columns=['mkt_carrier_fl_num', 'op_carrier_fl_num', 'origin_airport_id', 'dest_airport_id', 'dep_delay', 'month', 'fl_date_year',
#     'fl_date_week_number', 'crs_dep_time', 'crs_arr_time', 'actual_elapsed_time'])

In [53]:
df.columns

Index(['mkt_carrier_fl_num', 'op_carrier_fl_num', 'origin_airport_id',
       'dest_airport_id', 'dep_delay', 'arr_delay', 'crs_dep_time',
       'crs_arr_time', 'crs_elapsed_time', 'actual_elapsed_time', 'air_time',
       'distance', 'month', 'fl_date_year', 'fl_date_week_number',
       'mean_dep_delay_carrier_origin_month',
       'mean_arr_delay_carrier_origin_month',
       'mean_dep_delay_carrier_origin_week',
       'mean_arr_delay_carrier_origin_week',
       'mean_dep_delay_carrier_origin_date',
       'mean_arr_delay_carrier_origin_date',
       'mean_dep_delay_carrier_origin_date_t-1_week',
       'mean_arr_delay_carrier_origin_date_t-1_week',
       'mean_dep_delay_carrier_origin_date_t-1_week_week_number',
       'mean_arr_delay_carrier_origin_date_t-1_week_week_number',
       'origin_region_Midwest', 'origin_region_Northeast',
       'origin_region_South', 'origin_region_West', 'dest_region_Midwest',
       'dest_region_Northeast', 'dest_region_South', 'dest_region_West

In [54]:
# Create list with the columns to be scaled.
numeric_cols = [
       'air_time', 'arr_delay', 
       'crs_elapsed_time',
       'distance', 'mean_dep_delay_carrier_origin_month',
       'mean_arr_delay_carrier_origin_month',
       'mean_dep_delay_carrier_origin_week',
       'mean_arr_delay_carrier_origin_week',
       'mean_dep_delay_carrier_origin_date',
       'mean_arr_delay_carrier_origin_date',
       'mean_dep_delay_carrier_origin_date_t-1_week',
       'mean_arr_delay_carrier_origin_date_t-1_week',
       'mean_dep_delay_carrier_origin_date_t-1_week_week_number',
       'mean_arr_delay_carrier_origin_date_t-1_week_week_number'
       ]

# Create a list with the dummy/categorical variables:
cat_cols = [
       'origin_region_Midwest', 'origin_region_Northeast',
       'origin_region_South', 'origin_region_West', 'dest_region_Midwest',
       'dest_region_Northeast', 'dest_region_South', 'dest_region_West',
       'arr_hrs_ctg_Afternoon', 'arr_hrs_ctg_Evening', 'arr_hrs_ctg_Morning',
       'arr_hrs_ctg_Night', 'dep_hrs_ctg_Afternoon', 'dep_hrs_ctg_Evening',
       'dep_hrs_ctg_Morning', 'dep_hrs_ctg_Night',
       'day_of_week_Friday',
       'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday',
       'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday',
       'haul_length_long', 'haul_length_medium', 'haul_length_short', 'cluster_km'
       ]

In [55]:
# 2022-10-27 8:57 Here is the function to scale the data. Please read the function documentation. 
# This function has been saved to the midterm_functions.py file.
def scale_data(df, numeric_cols, cat_cols):
    """
    - Perform standardization (StandardScaler) on the numeric_cols of the dataframe. 
    - combines both numeric and categorical back to the entire feature dataframe.

    Params:
    - df: Dataframe object with both feature and target data. 
    - numeric_cols: Name of the numeric columns to be scaled.
    - cat_cols: Name of the categorical columns (dummy variables) NOT to be scaled.

    Returns: Dataframe with numeric data scaled and categorical data as-is.

    """
    # Create the scaler based on the training dataset
    scaler = StandardScaler()
    scaler.fit(df[numeric_cols])
    X_numeric_scaled = scaler.transform(df[numeric_cols])
    X_categorical = df[cat_cols].to_numpy()
    X = pd.DataFrame(np.hstack((X_categorical, X_numeric_scaled)), columns=cat_cols + numeric_cols)
    return X

In [56]:
# Call the function:
df_scaled = scale_data(df, numeric_cols, cat_cols)

In [57]:
df_scaled.columns

Index(['origin_region_Midwest', 'origin_region_Northeast',
       'origin_region_South', 'origin_region_West', 'dest_region_Midwest',
       'dest_region_Northeast', 'dest_region_South', 'dest_region_West',
       'arr_hrs_ctg_Afternoon', 'arr_hrs_ctg_Evening', 'arr_hrs_ctg_Morning',
       'arr_hrs_ctg_Night', 'dep_hrs_ctg_Afternoon', 'dep_hrs_ctg_Evening',
       'dep_hrs_ctg_Morning', 'dep_hrs_ctg_Night', 'day_of_week_Friday',
       'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday',
       'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday',
       'haul_length_long', 'haul_length_medium', 'haul_length_short',
       'cluster_km', 'air_time', 'arr_delay', 'crs_elapsed_time', 'distance',
       'mean_dep_delay_carrier_origin_month',
       'mean_arr_delay_carrier_origin_month',
       'mean_dep_delay_carrier_origin_week',
       'mean_arr_delay_carrier_origin_week',
       'mean_dep_delay_carrier_origin_date',
       'mean_arr_delay_carrier_origin

In [58]:
X = df_scaled.drop('arr_delay', axis = 1)   # features
y = df_scaled['arr_delay']    # labels

# Initialize the object 
lr = LinearRegression()

# Fit the model on our data
lr.fit(X, y)


print(lr.coef_)
lr.score(X,y)

[ 1.22465181e+08  1.22465181e+08  1.22465181e+08  1.22465181e+08
 -6.12005579e+09 -6.12005579e+09 -6.12005579e+09 -6.12005579e+09
  6.30418966e+09  6.30418966e+09  6.30418966e+09  6.30418966e+09
 -5.24706828e+09 -5.24706828e+09 -5.24706828e+09 -5.24706828e+09
  2.11300924e+10  2.11300924e+10  2.11300924e+10  2.11300924e+10
  2.11300924e+10  2.11300924e+10  2.11300924e+10  3.88269666e+10
  3.88269666e+10  3.88269666e+10  1.32604057e-02  2.74250113e+00
 -2.16160826e+00 -6.18466437e-01  5.04940940e+11  7.48403382e+11
 -1.26235235e+11 -1.87100845e+11 -1.26235235e+11 -1.87100845e+11
 -1.26235235e+11 -1.87100845e+11 -1.26235235e+11 -1.87100845e+11]


0.14549315626467918

In [59]:
from sklearn.linear_model import LinearRegression

par = {
    'normalize': [True, False]
}

lr = LinearRegression()
lr_attributes = supervised(X, y, lr, par, 'Liner_Regression')
best_lr = lr_attributes.get_best_model()


**Data has been scaled.**


/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
/opt/homebrew/Caskroom/miniforge/b


Liner_Regression evaluation metrics: 
	Test data	Training data		Difference
RMSE: 		0.92		0.85		0.07
MAE: 		0.65		0.65		-0.00
R^2: 		0.14		0.14		-0.00
Best model parameters from randomized search: {'normalize': True}


In [60]:
best_lr.coef_

array([ 9.76630506e+08,  8.48833702e+08,  1.23212871e+09,  1.14598086e+09,
       -3.17358801e+09, -2.75406471e+09, -4.00503739e+09, -3.72261656e+09,
        2.66416585e+10,  3.08305148e+10,  2.86934961e+10,  1.58422380e+10,
        8.25394736e+09,  9.15130171e+09,  9.55576632e+09,  1.98755491e+09,
       -1.16742536e+10, -1.27996910e+10, -1.18841146e+10, -1.26097933e+10,
       -1.22597414e+10, -1.26657510e+10, -1.25835157e+10,  3.31108878e+09,
        1.73867184e+10,  1.76165972e+10,  7.79831822e-03,  2.74263597e+00,
       -2.17068294e+00, -6.10214352e-01, -9.43097812e+10,  6.50626840e+10,
        2.35774453e+10, -1.62656710e+10,  2.35774453e+10, -1.62656710e+10,
        2.35774453e+10, -1.62656710e+10,  2.35774453e+10, -1.62656710e+10])

In [ ]:
# Save the file to CSV
from datetime import datetime
datetime_now = datetime.now().strftime('%Y-%m-%d_%H%M')
filename = f'Linear_Regression_{datetime_now}.sav'
print(filename)

In [61]:
# Save the model
pickle.dump(df,open(filename, 'wb'))